In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse, parse_qs
import re

# Load the mapping table
loc_dict = pd.read_csv("/content/naver_kin_regions_df_l2 (1).csv")
# Convert to dictionary for faster lookups
location_map = dict(zip(loc_dict['지역 코드'].astype(str), loc_dict['지명']))

def get_dirid(url):
    """Extract dirId from URL"""
    parsed_url = parse_qs(urlparse(url).query)
    dir_id = parsed_url.get('dirId', [None])[0]
    return dir_id

def get_docid(url):
    """Extract docId from URL"""
    parsed_url = parse_qs(urlparse(url).query)
    doc_id = parsed_url.get('docId', [None])[0]
    return doc_id

def get_location_name(dir_id):
    """Get location name from dirId using the mapping table"""
    if dir_id is None:
        return "Unknown Location"

    # Handle different dirId lengths
    if len(dir_id) >= 6:  # Full code
        return location_map.get(dir_id, "Location Not Found")
    elif len(dir_id) == 2:  # Just '12'
        return "전국"
    elif len(dir_id) == 4:  # e.g., '1201'
        # Try to find any matching location that starts with this prefix
        matching_locations = [name for code, name in location_map.items()
                            if code.startswith(dir_id)]
        if matching_locations:
            # Return the common prefix (e.g., "서울특별시")
            common_words = set(matching_locations[0].split()[0])
            return list(common_words)[0]

    return "Location Not Found"

def collect_information(url, Q_df, A_df):
    """Modified collect_information function to use the mapping table"""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract information as before
    title = soup.find(attrs={'class': 'endTitleSection'}).get_text(strip=True)
    title = title.replace(",", "")
    Q_detail = soup.find(attrs={'class': 'questionDetail'}).get_text(strip=True, separator=' ')
    Q_detail = Q_detail.replace(",", "")
    Q_info = soup.find_all(class_='infoItem')
    Q_view = Q_info[0].get_text()[3:]
    Q_date = Q_info[1].get_text()[3:]

    # Date handling
    if Q_date.endswith(" 전"):
        Q_date = datetime.today().strftime('%Y.%m.%d')
    if Q_date.startswith("작성일"):
        Q_date = Q_date.replace("작성일", "")

    # Get IDs and add to dataframe with location from mapping
    doc_ID = get_docid(url)
    dir_ID = get_dirid(url)

    # Create Q_inf_list with location information
    Q_inf_list = [doc_ID, title, Q_detail, dir_ID, Q_view, Q_date]

    # Handle answers
    ans = []
    A_dates = []
    try:
        answers = soup.find_all(class_='se-main-container')
        ans_date = soup.find_all(class_='answerDate')
        for answer in answers:
            ans.append(answer.get_text(strip=True).replace('\u200b', ''))
        for ans_dates in ans_date:
            A_dates.append(ans_dates.get_text(strip=True))
    except:
        pass

    # Create answer entries
    A_inf_list = []
    for idx, (answer, date) in enumerate(zip(ans, A_dates),1):

      #clean answer data
      answer_text = answer.replace(",", " ").strip()
      if answer_text.startswith('답변'):
          answer_text = answer_text[2:]
      #Handle Date
      if date.endswith(" 전"):
          date = datetime.today().strftime('%Y.%m.%d')
      #Create consistent answer Id format: docID_1, docID_2 etc.
      answer_id = f"{doc_ID}_{idx}"

      #Add to list
      A_inf_list.append([doc_ID, answer_id, answer_text, date])


    # Add to dataframes
    Q_df.loc[Q_df.shape[0]] = Q_inf_list
    for i in A_inf_list:
        A_df.loc[A_df.shape[0]] = i

    return Q_df, A_df

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time

def get_url_from_list(search_url):
    """Get all question URLs from a list page"""
    urls = []
    response = requests.get(search_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    elements = soup.find_all(class_=re.compile(r'^_nclicks:kls_new\.list'))

    for element in elements[1:]:
        href = "https://kin.naver.com" + element.get('href')
        urls.append(href)
    return urls

def extract_regional_data(region_code, num_pages=10):
    """Extract data for a specific region"""
    # Initialize dataframes
    Q_df = pd.DataFrame(columns=['doc_id', 'title', 'question', 'region_id', 'views', 'question_date'])
    A_df = pd.DataFrame(columns=['doc_id', 'answer_index', 'answer', 'answer_date'])

    urls = []
    # Collect URLs from all pages
    for i in range(1, num_pages + 1):
        page_url = f"https://kin.naver.com/qna/list.naver?dirId={region_code}&queryTime={datetime.today().strftime('%Y-%m-%d')}%2000%3A25%3A07&page={i}"
        urls.extend(get_url_from_list(page_url))
        time.sleep(0.5)  # Add delay to prevent too many requests

    # Process each URL
    for url in tqdm(urls, desc=f"Processing {region_code}", ncols=100):
        try:
            collect_information(url, Q_df, A_df)
            time.sleep(0.1)  # Small delay between requests
        except Exception as e:
            print(f"Error processing URL {url}: {str(e)}")
            continue

    return Q_df, A_df

def extract_by_region():
    # Load region mapping
    regions_df = pd.read_csv("/content/naver_kin_regions_df_l2 (1).csv",
                            names=['지명', '지역코드'])

    # Create output directory
    import os
    if not os.path.exists('regional_data'):
        os.makedirs('regional_data')

    # Process all regions
    for _, row in tqdm(regions_df.iterrows(), total=len(regions_df), desc="전체 지역 처리중"):
        지역코드 = str(row['지역코드'])
        지명 = row['지명']

        print(f"\n처리중: {지명} (코드: {지역코드})")

        try:
            # Extract data for this region
            Q_df, A_df = extract_regional_data(지역코드, num_pages=5)

            # Create region-specific directory using 지명
            지역경로 = f'regional_data/{지명}'
            if not os.path.exists(지역경로):
                os.makedirs(지역경로)

            # Save files
            Q_df.to_csv(f'{지역경로}/questions_{지역코드}.csv', index=False, encoding='utf-8-sig')
            A_df.to_csv(f'{지역경로}/answers_{지역코드}.csv', index=False, encoding='utf-8-sig')

            print(f"저장완료: {지명}")
            print(f"질문 수: {len(Q_df)}")
            print(f"답변 수: {len(A_df)}")

            time.sleep(1)  # Prevent server overload

        except Exception as e:
            print(f"에러 발생 - {지명} ({지역코드}): {str(e)}")
            continue

# Run extraction for all regions
extract_by_region()



전체 지역 처리중:   0%|          | 0/259 [00:00<?, ?it/s]


처리중: 지명 (코드: 지역 코드)



Processing 지역 코드: 100%|███████████████████████████████████████| 100/100 [01:05<00:00,  1.53it/s]


저장완료: 지명
질문 수: 100
답변 수: 46


전체 지역 처리중:   0%|          | 1/259 [01:11<5:08:28, 71.74s/it]

...

전체 지역 처리중: 100%|█████████▉| 258/259 [2:13:25<00:05,  5.90s/it]


처리중: 세종특별자치시 도담동 (코드: 121724)



Processing 121724: 100%|██████████████████████████████████████████████| 1/1 [00:00<00:00,  1.94it/s]


저장완료: 세종특별자치시 도담동
질문 수: 1
답변 수: 0


전체 지역 처리중: 100%|██████████| 259/259 [2:13:32<00:00, 30.94s/it]


In [14]:
!zip -r regional_data.zip regional_data

from google.colab import files
files.download('regional_data.zip')

  adding: regional_data/ (stored 0%)
  adding: regional_data/부산광역시 기장군/ (stored 0%)
  adding: regional_data/부산광역시 기장군/questions_120218.csv (deflated 51%)
  adding: regional_data/부산광역시 기장군/answers_120218.csv (deflated 60%)
  adding: regional_data/전라남도 고흥군/ (stored 0%)
  adding: regional_data/전라남도 고흥군/questions_121207.csv (deflated 52%)
  adding: regional_data/전라남도 고흥군/answers_121207.csv (deflated 46%)
  adding: regional_data/충청남도 서산시/ (stored 0%)
  adding: regional_data/충청남도 서산시/questions_121505.csv (deflated 58%)
  adding: regional_data/충청남도 서산시/answers_121505.csv (deflated 56%)
  adding: regional_data/경상남도 사천시/ (stored 0%)
  adding: regional_data/경상남도 사천시/questions_121005.csv (deflated 47%)
  adding: regional_data/경상남도 사천시/answers_121005.csv (deflated 55%)
  adding: regional_data/경기도 용인시/ (stored 0%)
  adding: regional_data/경기도 용인시/answers_120923.csv (deflated 62%)
  adding: regional_data/경기도 용인시/questions_120923.csv (deflated 62%)
  adding: regional_data/경기도 파주시/ (stored 0%)
  adding

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# For specific regions only
    regions_of_interest = ['120116',
                           ]  # Example: 서울 강동구

    for region_code in regions_of_interest:
      Q_df, A_df = extract_regional_data(region_code, num_pages=5)
      Q_df.to_csv(f'data/{region_code}_questions.csv', index=False)
      A_df.to_csv(f'data/{region_code}_answers.csv', index=False)

    # Save all data to a single file

    all_questions = pd.concat([data['questions'] for data in regional_data.values()])
    all_answers = pd.concat([data['answers'] for data in regional_data.values()])

    all_questions.to_csv('data/all_questions.csv', index=False)
    all_answers.to_csv('data/all_answers.csv', index=False)